
# Quick Start Demo: Environment-Centered Explanations (LIME · SHAP · COSE)

This notebook gives you a **5-minute, minimal demo** that:
1. Generates a tiny grid environment
2. Runs one classical planner (**A\***)
3. Produces environment-centered explanations with **LIME**, **SHAP**, and **COSE**
4. Renders example figures and previews evaluation CSVs

> **Prerequisites**
> - You have cloned this repo locally
> - You have installed dependencies: `pip install -r requirements.txt`
> - You can run the CLI modules in `cli/` (e.g., `python -m cli.run_eval --help`)

If your repository lives elsewhere, update `REPO_DIR` below.


In [ ]:

import os, sys, glob, subprocess, json
from pathlib import Path

# ---- Configure paths ----
REPO_DIR = Path.cwd()  # change this if needed
RESULTS_DIR = REPO_DIR / "results" / "quickstart"
ENVS_DIR = RESULTS_DIR / "envs"
CSV_DIR = RESULTS_DIR / "csv"
FIGS_DIR = RESULTS_DIR / "figs"

for d in (RESULTS_DIR, ENVS_DIR, CSV_DIR, FIGS_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Repository:", REPO_DIR)
print("Results dir:", RESULTS_DIR)


## 1) Generate a tiny environment

In [ ]:

# We generate just a couple of tiny environments for a fast demo.
# Adjust sizes/densities if your CLI expects different flags.
cmd = [
    sys.executable, "-m", "cli.generate_envs",
    "--outdir", str(ENVS_DIR),
    "--sizes", "20",
    "--densities", "0.2",
    "--n-per-config", "2"
]
print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True)
print("Done. Files:", len(glob.glob(str(ENVS_DIR / "E_*.npz"))))


## 2) Run a tiny evaluation (A*, LIME, SHAP, COSE)

In [ ]:

# We pick a single planner and three explainers to keep this fast.
# If your run_eval supports limiting the number of envs, keep it tiny.
env_glob = str(ENVS_DIR / "E_*.npz")
cmd = [
    sys.executable, "-m", "cli.run_eval",
    "--env-glob", env_glob,
    "--planners", "a_star",
    "--explainers", "lime", "shap", "cose",
    "--outdir", str(CSV_DIR),
]
print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True)
print("Done. CSV files:", len(glob.glob(str(CSV_DIR / "eval_*.csv"))))


## 3) Make figures

In [ ]:

cmd = [
    sys.executable, "-m", "cli.make_figs",
    "--eval-glob", str(CSV_DIR / "eval_*.csv"),
    "--outdir", str(FIGS_DIR),
]
print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True)
print("Wrote figures to:", FIGS_DIR)


## 4) Preview evaluation CSVs

In [ ]:

import pandas as pd
csv_files = sorted(glob.glob(str(CSV_DIR / "eval_*.csv")))
if not csv_files:
    raise SystemExit("No CSVs found. Check the previous steps' output.")

df = pd.concat([pd.read_csv(p) for p in csv_files], ignore_index=True)
print("Rows:", len(df), "| Columns:", list(df.columns))
df.head(20)


## 5) Display generated figures (if available)

In [ ]:

from IPython.display import display, Image

figure_patterns = [
    "q1_success_at_k.pdf",
    "q2_min_set_size*.pdf",
    "q2_min_set_size_violin.pdf",
    "q3_*.pdf",
    "*.png",
    "*.pdf",
]
found = []
for pat in figure_patterns:
    found += glob.glob(str(FIGS_DIR / pat))

if not found:
    print("No figures found in", FIGS_DIR)
else:
    print("Showing up to 8 figures...")
    for i, p in enumerate(sorted(found)[:8]):
        print(p)
        display(Image(filename=p) if p.lower().endswith((".png", ".jpg", ".jpeg")) else p)



---

### Troubleshooting

- **Module not found**: Ensure you're running the notebook from the repo root or add `REPO_DIR` to `sys.path`.
- **CLI signature mismatch**: Your local CLI flags may differ. Run `python -m cli.generate_envs --help`, `python -m cli.run_eval --help`, and `python -m cli.make_figs --help` and adjust arguments.
- **No figures produced**: Some figures require specific datasets. Start with `q1_success_at_k` and `q2_min_set_size` outputs, or reduce the environment count.
- **Performance**: This demo is intentionally small. For the full paper-scale evaluation, increase `--sizes`, `--densities`, and number of environments.

---

### Next Steps

- Explore `results/quickstart/csv/` for metric CSVs.
- Inspect `results/quickstart/figs/` for figures used in the paper.
- Extend planners (`--planners a_star dijkstra ...`) and affordances; add robustness and faithfulness analyses.
